to do:
train_X :
- change label to 0/1 (Normal/Attack)
- Normalize data
- Split to matrix (60 * var_length) = 60 because 1 hour
- transpose matrix to (var_lenth * 60)

train_Y :
- split to matrix @60
- if in the matrix there is 1 (attack) make label as 1 

Input shape:
(49990, 53)
Expected output shape:
train_X: (7498, 52, 60)
train_Y: (7498, 1)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
data = pd.read_csv('pca_result.csv')
# df = data['Timestamp','FIT101','LIT101','MV101','P101','P102','AIT201','AIT202','AIT203','FIT201','MV201','P201','P202','P203','P204','P205','P206','DPIT301','FIT301','LIT301','MV301','MV302','MV303','MV304','P301','P302','AIT401','AIT402','FIT401','LIT401','P401','P402','P403','P404','UV401','AIT501','AIT502','AIT503','AIT504','FIT501','FIT502','FIT503','FIT504','P501','P502','PIT501','PIT502','PIT503','FIT601','P601','P602','P603','Normal/Attack']
data.info

<bound method DataFrame.info of         Unnamed: 0         p_0        p_1        p_2       p_3        p_4  \
0                0 -127.800274 -54.154816 -47.741916 -9.507443  51.966193   
1                1 -127.768803 -54.123383 -47.738719 -9.455319  51.967693   
2                2 -127.625772 -54.303811 -47.982165 -9.325753  51.963625   
3                3 -127.479695 -54.327131 -48.288432 -9.243717  51.951160   
4                4 -127.598297 -53.855411 -48.800676 -9.379561  51.936331   
...            ...         ...        ...        ...       ...        ...   
449914      449914 -141.318386 -58.427159 -66.254251 -9.280435 -46.508155   
449915      449915 -141.202033 -57.611166 -66.615572 -9.485075 -46.515932   
449916      449916 -141.234248 -57.227585 -66.427084 -9.571052 -46.518612   
449917      449917 -141.544191 -57.073277 -66.506375 -9.727996 -46.517102   
449918      449918 -141.437740 -56.770660 -66.817153 -9.719421 -46.521181   

             p_5       p_6       p_7       

In [11]:
data.head()

,Unnamed: 0,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,...,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,label
0,0,-127.800274,-54.154816,-47.741916,-9.507443,51.966193,-1.021234,2.048087,-0.628924,-2.053950,...,0.256739,-0.171998,-0.018811,-0.029206,0.018685,-0.045521,-0.009932,0.003885,-0.016100,0
1,1,-127.768803,-54.123383,-47.738719,-9.455319,51.967693,-0.999320,2.060894,-0.632075,-2.052076,...,0.261789,-0.173308,-0.020864,-0.029728,0.018684,-0.038593,-0.010118,0.004012,-0.015703,0
2,2,-127.625772,-54.303811,-47.982165,-9.325753,51.963625,-0.999087,2.067530,-0.634220,-2.018816,...,0.259136,-0.174245,-0.019789,-0.030218,0.017814,-0.023013,-0.011533,0.003967,-0.015316,0
3,3,-127.479695,-54.327131,-48.288432,-9.243717,51.951160,-1.031954,1.983926,-0.633737,-1.994673,...,0.253910,-0.173540,-0.018646,-0.029524,0.016522,-0.006551,-0.011410,0.003876,-0.015194,0
4,4,-127.598297,-53.855411,-48.800676,-9.379561,51.936331,-1.059339,1.856140,-0.635268,-1.966480,...,0.246103,-0.171918,-0.016312,-0.028308,0.014878,0.006208,-0.010750,0.003772,-0.014879,0


- change label to 0/1 (Normal/Attack)

In [12]:
# data['label'] = pd.factorize(data['Normal/Attack'])[0].astype(np.uint16)
data['label'].unique()

array([0, 1], dtype=int64)

- change timestamp to datetime format

In [6]:
# separate the date
# train_dates = pd.to_datetime(data[' Timestamp'])

In [13]:
# separate the label
train_label = data['label']
#drop timestamp, label, and ex label column
data = data.drop(['label', 'Unnamed: 0'], axis=1)
# change type to float
data = data.astype(float, errors = 'raise')
data.shape

(449919, 25)

In [14]:
# normalize the data
scaler = StandardScaler()
scaler = scaler.fit(data)
df_for_training_scaled = scaler.transform(data)

In [15]:
# delete last 39 rows, because we want to reshape to 60 length 
df_for_training_scaled = df_for_training_scaled[:-39, :]
df_for_training_scaled.shape

(449880, 25)

In [16]:
# Transpose data before splitting
transposed_data = df_for_training_scaled.T
transposed_data.shape

(25, 449880)

In [17]:
splitted_data = np.hsplit(transposed_data,7498) #equally 60 length
X_train = np.asarray(splitted_data)
X_train.shape

(7498, 25, 60)

now create the label train_Y

In [18]:
train_label = np.asarray(train_label)
train_label = train_label[:-39] #remove last 39 labels
splitted_train_label = np.split(train_label,7498) #split to equally 60
yarr = np.asarray(splitted_train_label)
yarr.shape

(7498, 60)

In [19]:
def linear_search(values, search_for):
   search_at = 0
   search_res = False
# Match the value with each data element	
   while search_at < len(values) and search_res is False:
      if values[search_at] == search_for:
         search_res = True
      else:
         search_at = search_at + 1
   return search_res

In [20]:
Y_train = []
for i in range(len(yarr)):
    attacked = linear_search(yarr[i], 1) #if we find 1 in 60 label (1 hour), true = attacked, false = normal
    Y_train.append(attacked)

Y_train = np.array(Y_train).astype('uint8')

In [21]:
Y_train = np.reshape(Y_train, (len(Y_train), 1)) #reshape to match the input for cnn
Y_train.shape

(7498, 1)

In [22]:
train_X, test_X, train_Y, test_Y = train_test_split(X_train, Y_train, test_size=0.25, random_state=42, shuffle=False)

In [23]:
print("train_X shape = ", train_X.shape)
print("train_Y shape = ", train_Y.shape)
print("test_X shape = ", test_X.shape)
print("test_Y shape = ", test_Y.shape)

train_X shape =  (5623, 25, 60)
train_Y shape =  (5623, 1)
test_X shape =  (1875, 25, 60)
test_Y shape =  (1875, 1)


In [24]:
# save numpy array as npy file
from numpy import save
# save to npy file
save('X_train.npy', train_X)
save('y_train.npy', train_Y)
save('X_test.npy', test_X)
save('y_test.npy', test_Y)